# Análisis de Redes Sociales (SNA)

## Análisis de los Episodios Nacionales: Primera Serie de Galdós

**Autora:** Alina Rojas

**Fecha de creación:** 2024-05-15

**Última fecha de modificación:** 2024-05-17

En este notebook se prepara y limpia un conjunto de datos textuales para la extracción de nombres de personajes utilizando modelos avanzados de procesamiento de lenguaje natural (spaCy y BERT). Los datos procesados serán utilizados posteriormente para construir y analizar redes de relaciones entre personajes en R, facilitando un análisis detallado de las interacciones y conexiones en la primera serie de los Episodios Nacionales de Benito Pérez Galdós.

## 1 Preparación del entorno

### 1.1 Instalación de paquetes

In [1]:
!pip install spacy
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 80.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


### 1.2 Importación de paquetes

In [33]:
# Importaciones generales y manipulación de datos
import pandas as pd
from itertools import combinations
import csv

# Importación para la interacción con Google Colab
from google.colab import files

# Procesamiento de texto y NLP
import spacy
from spacy.matcher import Matcher
import re

# Descarga y configuración de stopwords para NLP con NLTK
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Modelos de lenguaje y herramientas de transformadores para NLP
from transformers import pipeline
from tqdm import tqdm

# Herramientas para procesamiento concurrente
from concurrent.futures import ProcessPoolExecutor

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### 1.3 Cargado de los datos

In [3]:
# Cargar el archivo CSV
books_metadata_filtered = pd.read_csv('books_metadata_filtered.csv')

In [4]:
# Ver estructura y detalles del dataframe
books_metadata_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63617 entries, 0 to 63616
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   text         63617 non-null  object
 1   book         63617 non-null  object
 2   line_number  63617 non-null  int64 
 3   chapter      63617 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 1.9+ MB


In [5]:
# Ver parte del contenido del dataframe
books_metadata_filtered.head()

,text,book,line_number,chapter
0,Se me permitirá que antes de referir el gran s...,Trafalgar,7,1
1,"diga algunas palabras sobre mi infancia, expli...",Trafalgar,8,1
2,manera me llevaron los azares de la vida a pre...,Trafalgar,9,1
3,catástrofe de nuestra marina.,Trafalgar,10,1
4,"Al hablar de mi nacimiento, no imitaré a la ma...",Trafalgar,11,1


### 1.4 Preparación de los datos

A continuación, se va a eliminar las stopwords del campo `text` para la extracción de tópicos, junto con los carácteres especiales.

In [6]:
# Cargar la lista de stopwords en español
spanish_stopwords = set(stopwords.words("spanish"))

def remove_stopwords_from_text(text):
    """
    Elimina las stopwords de un texto dado.

    Parámetros:
    text (str): El texto del cual se eliminarán las stopwords.

    Retorna:
    str: Una cadena de texto con las stopwords eliminadas.

    Nota:
    Esta función asume que existe una lista 'spanish_stopwords' que contiene las stopwords en español.
    """
    words = text.split()  # Divide el texto en palabras.
    # Filtra las palabras, eliminando las que están en la lista de stopwords.
    filtered_words = [word for word in words if word.lower() not in spanish_stopwords]
    # Une las palabras filtradas en una sola cadena de texto y la retorna.
    return ' '.join(filtered_words)

In [7]:
# Reemplazar los signos de puntuación en la columna 'text', manteniendo tildes y diéresis, y eliminar stopwords
books_metadata_filtered['text_filtered'] = books_metadata_filtered['text'].str.replace(r'[-_]', ' ', regex=True).apply(remove_stopwords_from_text)

In [8]:
# Ver parte del contenido del dataframe
books_metadata_filtered.head()

,text,book,line_number,chapter,text_filtered
0,Se me permitirá que antes de referir el gran s...,Trafalgar,7,1,"permitirá referir gran suceso testigo,"
1,"diga algunas palabras sobre mi infancia, expli...",Trafalgar,8,1,"diga palabras infancia, explicando extraña"
2,manera me llevaron los azares de la vida a pre...,Trafalgar,9,1,manera llevaron azares vida presenciar terrible
3,catástrofe de nuestra marina.,Trafalgar,10,1,catástrofe marina.
4,"Al hablar de mi nacimiento, no imitaré a la ma...",Trafalgar,11,1,"hablar nacimiento, imitaré mayor parte"


## 2 Análisis de Redes Sociales (SNA)

### 2.1 Preparación de los datos de grafo

Para extraer nombres de personajes de manera precisa y robusta, hemos decidido utilizar dos herramientas avanzadas de procesamiento de lenguaje natural (NLP): spaCy y BERT (a través de la biblioteca Hugging Face).

**spaCy** es una de las bibliotecas más destacadas para NLP, conocida por su rapidez, precisión y capacidad para manejar grandes volúmenes de texto de manera eficiente. SpaCy proporciona herramientas poderosas para el reconocimiento de entidades nombradas (NER), que es crucial para identificar y extraer nombres de personajes de los textos. Las ventajas de utilizar spaCy en nuestro proyecto incluyen:

- **Reconocimiento preciso de entidades:** SpaCy dispone de modelos entrenados que pueden identificar nombres de personas, lugares y organizaciones con alta precisión.
- **Facilidad de integración:** SpaCy es fácil de integrar con otras herramientas y bibliotecas de Python, lo que permite aplicar su funcionalidad directamente sobre nuestros datos.
- **Rendimiento:** SpaCy está diseñado para ser rápido y eficiente, ideal para procesar grandes volúmenes de texto sin sacrificar precisión.

Para complementar la capacidad de spaCy y aumentar la precisión en la identificación de entidades, también implementamos un **pipeline de NER utilizando BERT** proporcionado por Hugging Face. BERT es un modelo de lenguaje profundo preentrenado que ha demostrado ser extremadamente efectivo en tareas de NLP debido a su capacidad para entender el contexto del lenguaje a un nivel más profundo que los enfoques tradicionales. La integración de BERT nos permite:

- **Filtrar y refinar las entidades:** A través del pipeline de NER de BERT, podemos filtrar y refinar los nombres identificados inicialmente por spaCy, asegurando que solo los nombres de personas relevantes y correctamente identificados sean utilizados en nuestras análisis posteriores.
- **Mejorar la robustez del reconocimiento:** BERT ayuda a manejar casos más complejos y ambiguos en la identificación de nombres, lo que refuerza la confiabilidad de nuestro proceso de extracción de datos.

Utilizando spaCy y BERT en conjunto, hemos establecido un flujo de trabajo robusto y eficiente para el procesamiento de textos, asegurando que los nombres de personajes se extraigan con la máxima precisión posible, preparando así los datos de manera óptima para su uso en la construcción de grafos de relaciones de personajes.

In [9]:
# Cargar el modelo preentrenado de spaCy en español
nlp = spacy.load('es_core_news_sm')

# Definir el pipeline de NER de Hugging Face
ner_pipeline = pipeline("ner", model="dslim/bert-base-NER")

def extract_person_names_spacy(text):
    """
    Extrae nombres de personas de un texto utilizando spaCy.

    Parámetros:
    text (str): Texto del cual se desean extraer nombres de personas.

    Retorna:
    list: Lista de nombres de personas extraídos del texto.
    """
    doc = nlp(text)
    person_names = [ent.text for ent in doc.ents if ent.label_ == 'PER']
    return person_names

def filter_person_names_bert(names, text):
    """
    Filtra nombres de personas utilizando el pipeline de NER de BERT.

    Parámetros:
    names (list): Lista de nombres extraídos previamente.
    text (str): Texto completo del cual se desean filtrar los nombres.

    Retorna:
    list: Lista de nombres de personas filtrados usando el modelo BERT.
    """
    ner_results = ner_pipeline(text)
    bert_person_names = [entity['word'] for entity in ner_results if entity['entity'] == 'B-PER']
    filtered_names = [name for name in names if name in bert_person_names or name in text]
    return filtered_names

def process_single_text(text):
    """
    Procesa un solo texto para extraer y filtrar nombres de personas.

    Parámetros:
    text (str): Texto a procesar.

    Retorna:
    list: Lista de nombres de personas filtrados.
    """
    # Extraer nombres con spaCy
    names_spacy = extract_person_names_spacy(text)
    # Filtrar los nombres extraídos por spaCy usando BERT
    filtered_names = filter_person_names_bert(names_spacy, text)
    return filtered_names

def process_text_column_concurrently(texts):
    """
    Procesa una columna de textos en paralelo para extraer y filtrar nombres de personas.

    Parámetros:
    texts (list): Lista de textos a procesar.

    Retorna:
    list: Lista de listas, donde cada sublista contiene nombres de personas filtrados de cada texto.
    """
    with ProcessPoolExecutor() as executor:
        all_person_names = list(tqdm(executor.map(process_single_text, texts), total=len(texts), desc="Processing texts"))
    return all_person_names


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

#### 2.1.1 Procesar el texto

In [11]:
# Aplicar la función a cada elemento de la columna 'text_filtered'
books_metadata_filtered['person_names'] = process_text_column_concurrently(books_metadata_filtered['text_filtered'])

print(books_metadata_filtered)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Processing texts: 100%|██████████| 63617/63617 [2:10:48<00:00,  8.11it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()



                                                    text  \
0      Se me permitirá que antes de referir el gran s...   
1      diga algunas palabras sobre mi infancia, expli...   
2      manera me llevaron los azares de la vida a pre...   
3                          catástrofe de nuestra marina.   
4      Al hablar de mi nacimiento, no imitaré a la ma...   
...                                                  ...   
63612  vigorosas; si os halláis imposibilitados para ...   
63613  generosos impulsos del pensamiento y las leyes...   
63614  Gabriel Araceli, que nació sin nada y lo tuvo ...   
63615                             Febrero-marzo de 1875.   
63616                FIN DE «LA BATALLA DE LOS ARAPILES»   

                             book  line_number  chapter  \
0                       Trafalgar            7        1   
1                       Trafalgar            8        1   
2                       Trafalgar            9        1   
3                       Trafalgar          

Observamos que parte de las palabras detectadas como nombres propios de personas, se encuentran expresiones verbales y nombres de lugares. Para deshacernos de las primeras, vamos a eliminar aquellos registros que empiecen por minúsculas.

In [24]:
# Función para limpiar nombres
def clean_name(name):
    """
    Limpia y filtra nombres eliminando signos de exclamación y aquellos que comienzan con minúscula.

    Parámetros:
    name (str): Nombre a limpiar y filtrar.

    Retorna:
    str or None: Nombre limpio si pasa los filtros, de lo contrario None.
    """
    # Eliminar signos de exclamación y otros caracteres no deseados
    name = re.sub(r'[!?]', '', name)
    # Filtrar nombres que empiezan con minúscula
    if name and name[0].isupper():
        return name
    return None

# Función para procesar la columna person_names
def process_person_names(person_names):
    """
    Procesa y limpia una lista de nombres de personas.

    Parámetros:
    person_names (list or str): Lista de nombres de personas o string representando una lista.

    Retorna:
    list: Lista de nombres de personas únicos y limpios.
    """
    if isinstance(person_names, str):
        person_names = eval(person_names)
    cleaned_names = [clean_name(name) for name in person_names if clean_name(name)]
    return list(set(cleaned_names))

# Asegurar que la columna person_names contiene listas
books_metadata_filtered['person_names'] = books_metadata_filtered['person_names'].apply(lambda x: eval(x) if isinstance(x, str) else x)

# Aplicar la limpieza y el filtrado a la columna person_names
books_metadata_filtered['person_names'] = books_metadata_filtered['person_names'].apply(process_person_names)


In [28]:
# Inspeccionar algunas filas para entender el formato de 'person_names'
print(books_metadata_filtered['person_names'].tail())

63612                   []
63613                   []
63614    [Gabriel Araceli]
63615                   []
63616                   []
Name: person_names, dtype: object


### 2.2 Exportar los datos

In [34]:
# Guardar el resultado en un nuevo archivo CSV
books_metadata_filtered.to_csv('books_metadata_filtered_characters.csv', index=False, quotechar='"', quoting=csv.QUOTE_NONNUMERIC)

# Descargar el archivo resultante
files.download('books_metadata_filtered_characters.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>